# LinkedIn Market Analysis


### Process:
1. Scrape data from Linkedin and Glassdoor, using Selenium.
2. EDA, cleaning, and export to CSV.
3. Compare to previous years' data (2 and 4 years ago) using tableau.

### Additional Datasets:
- 2018: 
- 2020: 
- 2022: [my submission to Kaggle]

### Reference Notebooks:
- https://www.kaggle.com/code/gawainlai/us-data-science-job-salary-regression-w-visuals (beyond my skill level)
- https://www.kaggle.com/code/discdiver/the-most-in-demand-skills-for-data-scientists (top skills)

## Import Libraries

In [1]:
# selenium imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

# web scraping imports
from bs4 import BeautifulSoup
import requests

# database imports
import re as re
import time
import pandas as pd
import os
import numpy as np

# import and load file to login to LinkedIn
from dotenv import load_dotenv
load_dotenv()

True

## Scrape LinkedIn Using the Selenium Driver

In [2]:
# initialize the LinkedIn scrape

# Options
options = webdriver.ChromeOptions() # init for chrome
options.add_argument('--incognito') # runs chrome in a 'clean slate' window
#options.add_argument('--headless') # runs chromedriver in the background, without opening a window

# Initialize the selenium driver
driver = webdriver.Chrome(options = options, executable_path='./chromedriver')
login_url = "https://www.linkedin.com/uas/login"

# Start the page
driver.get(login_url)
time.sleep(3)

# Target the login elements
email = driver.find_element("id", "username")
password = driver.find_element("id", "password")

# Load env variables
my_email = os.getenv("linkedin_username")
my_password = os.getenv("linkedin_password")

# Input in the form
email.send_keys(my_email)
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

/var/folders/yx/4xsyrpbs305132t_87mnd3600000gn/T/ipykernel_80162/3666220930.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options = options, executable_path='./chromedriver')


In [3]:
def scrape_links(data_role):
    """ Scrape 40 pages of a LinkedIn job search for job links, using the given data role as the search term """
    
    # SCRAPE 40 PAGES
    for i in range(2): # FIX: set 2 to 40 for final product
        print(f'running search for {i+1} of 40 pages for {data_role}.')
        
        # navigate to the correct page
        scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={i*25}"
        if i == 0:
            scrape_url = f"https://www.linkedin.com/jobs/search/?&keywords={data_role}&refresh=true&start={1}"
        driver.get(scrape_url)
        time.sleep(5)

        # SCROLL DOWN THE JOB LIST # FIX: can't get selenium to work
        #driver.execute_script("window.scrollTo(0, document.body.div.jobs-search-results-list.scrollHeight);")
        #left_rail_bottom = driver.find_element(By.CLASS_NAME, "global-footer-compact__content.t-12.t-normal.text-align-center.clear-both.compactfooter-copyright")
        #left_rail_bottom = driver.find_element(By.CSS_SELECTOR, "li.global-footer-compact__item")
        #left_rail_bottom = driver.find_element(By.ID, "compactfooter-about")
        #driver.execute_script("arguments[0].scrollIntoView();", left_rail_bottom)
        # scroll down for each job element
        #driver.execute_script("arguments[0].scrollIntoView();", job)
        time.sleep(1)
        
        # convert page text to beautiful soup
        src = driver.page_source
        soup_for_page = BeautifulSoup(src, 'lxml')
        
        # create a list of jobs on the current page, to iterate through after each scrape
        job_links = []
        jobs_on_page = soup_for_page.find_all("a", attrs={"class":"disabled ember-view job-card-container__link job-card-list__title"})
        for k in jobs_on_page: # length of jobs varies by page
            job_links.append(k["href"])
        print(f'total job links from page {i+1}:', len(job_links)) # DEBUG
        #print(f'job links from page {i+1}:',job_links) # DEBUG
        
    return job_links

In [4]:
def see_more():
    """ Load an entire LinkedIn job post page """
    
    # Click on the 'see more' button on a LinkedIn job post (there are several window types)
    try:
        driver.find_element_by_class_name("artdeco-card__actions").click()
        print('find_element_by_class_name("artdeco-card__actions") SUCCESS')
    except:
        try:
            driver.find_element(By.Class, "show-more-less-html__button show-more-less-html__button--more").click()
            print('driver.find_element(By.Class, "show-more-less-html__button ... SUCCESS')
        except:
            print('Find another solution to SEE MORE on this page')
                            
    # Scroll down the entire page for easier data collection
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    
    time.sleep(1)

In [15]:
def scrape_listing(links):
    """ Returns all scraped data for each job listing from the links passed into the function """

    # VARIABLE ASSIGNMENT create lists to store all scraped data (10 criteria)
    titles, companies, locations, remote, post_dates, num_applicants, contract, size, desc, salaries = [], \
        [], [], [], [], [], [], [], [], []
    
    # SCRAPE ALL LINKS scrape all jobs on the current page using passed in links
    for idx, link in enumerate(links):
        print(f'\nScraping job {idx} of {len(links)}.') # DEBUG TEXT
        
        # GO TO PAGE Navigate to page
        #print('\n\n\nkey:', key, '\nvalue:', value[0][0]) # DEBUG TEXT
        driver.get(f'https://linkedin.com{link}')
        time.sleep(5)
        
        # SEE FULL PAGE click 'see more' and scroll down
        see_more()
        
        # BEAUTFUL SOUP EXTRACTION convert page text to beautiful soup
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')
        
        # DATA COLLECTION return data of results on current selected sub-page to new lists
        
        # TITLE
        try:
            titles.append(soup.select('h1.t-24.t-bold.jobs-unified-top-card__job-title')[0].get_text().replace('\n','').strip())
        except:
            print('title could not be found')
            titles.append(None)
         
        # COMPANY
        try:
            companies.append(soup.select('span.jobs-unified-top-card__company-name')[0].get_text().replace('\n','').strip())
        except:
            print('company could not be found')
            companies.append(None)
            
        # LOCATION
        try:
            locations.append(soup.select('span.jobs-unified-top-card__bullet')[0].get_text().replace('\n','').strip())
        except:
            print('location could not be found')
            locations.append(None)
        
        # REMOTE POSITION
        try:
            remote.append(soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text().replace('\n','').strip())
        except:
            # FIX: check the description using regex. ie) "100% remote work" or "Remote working optional". Capture the whole sentence, from . to remote to .? From one special character to the next? Or just two words before and after, check the unique values, and narrow down from there.
            print('remote position could not be found')
            remote.append(None)
            
        # POST DATE
        try:
            post_dates.append(soup.select('span.jobs-unified-top-card__posted-date')[0].get_text().replace('\n','').strip())
        except:
            print("could not find 'posted date'")
            post_dates.append(None)
        
        # NUMBER of APPLICANTS
        try:
            num_applicants.append(soup.select('span.jobs-unified-top-card__applicant-count')[2].get_text().replace('\n','').strip())
        except:
            try:
                num_applicants.append(soup.select('span.jobs-unified-top-card__bullet')[1].get_text().replace('\n','').strip())
                print("could not find 'number of applicants' in applicant count")
            except:
                print("could not find 'number of applicants' in applicant count or bullet")
                num_applicants.append(None)
        
        # FULL TIME
        try:
            contract.append(soup.select('li.jobs-unified-top-card__job-insight.span').get_text()
                .replace('\n','').strip().rsplit(' ', 1)[-1])
        except:
            try:
                contract.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                    .replace('\n','').strip().rsplit(' ', 1)[-1])
                print('could not find "contract type" in job insights.span')
            except:
                print("could not find 'contract type' in job insights.span or job insights")
                contract.append(None)
        
        # COMPANY SIZE
        try:
            size.append(soup.select('li.jobs-unified-top-card__job-insight')[1].get_text().replace('\n','').strip())
        except:
            print("could not find 'company size'")
            size.append(None)
        
        # DESCRIPTION
        try:
            desc.append(soup.select('div.jobs-box__html-content.jobs-description-content__text.t-14.t-normal.jobs-description-content__text--stretch')[0].get_text().strip())
        except:
            print("could not find description (probably shouldn't apply!)")
            
        # SALARY
        try:
            salaries.append(soup.select('a.app-aware-link')[6].get_text().replace('\n','').strip())
            if '$' not in salaries[-1]:
                salaries.pop()
                salaries.append(None)
        except:
            try:
                salaries.append(soup.select('p.t-16')[0].get_text().replace('\n','').strip())
                print("could not find 'salary' with #SALARY tag")
            except:
                try:
                    salaries.append(re.find('($.)', desc).replace('\n','').strip())
                    print("could not find 'salary' with #SALARY tag or in p.t-16")
                except:
                    try: 
                        salaries.append(soup.select('li.jobs-unified-top-card__job-insight')[0].get_text()
                            .replace('\n','').strip().rsplit(' ', 1)[0].rstrip(' ·'))
                        print("could not find 'salary' with #SALARY tag, in p.t-16, or in description")
                    except:
                        print("could not find 'salary' with #SALARY tag, in p.t-16, in description, or in full-time")
                        salaries.append(None)

    # DICTIONARY ASSIGNMENT pass data from lists into a dictionary
    dict_from_scrape = {'title':titles, 'company':companies, 'location':locations, 'remote':remote, 
                        'post_date':post_dates, 'num_applicants':num_applicants, 'contract_type':contract, 
                        'company_size':size, 'description':desc, 'salary':salaries}

    # DATAFRAME ASSIGNMENT
    df_from_scrape = pd.DataFrame(dict_from_scrape)

    return df_from_scrape

In [6]:
def clean_list(list_):
    list_flattened = [a for y in list_ for a in y]
    return list(set(list_flattened))

In [7]:
data_roles = ['data analyst', 'data scientist', 'data architect', 'business analyst', 'finance analyst', 
              'data engineer', 'marketing analyst', 'data warehouse analyst', 'data management analyst'] # Titles taken from previous market analysis, used to capture all links to scrape with matching search term. # FIX: add all roles

job_links = [] # init list to capture all job links
#df = pd.DataFrame() # init dataframe to capture all scraped jobs # FIX: might not need

# for each role title given above, return a list of up to 1,000 jobs from LinkedIn
for title in data_roles:
    job_links.append(scrape_links(title.replace(' ','%20')))
#print(job_links) # DEBUG

job_links_cleaned = clean_list(job_links) # DUPLICATES remove dupliate links before scraping

# Scrape each new link
#df = pd.concat([df, scrape_listing(job_links_cleaned)], axis=0).reset_index(drop=True) # FIX: might not need
df = pd.DataFrame(scrape_listing(job_links_cleaned))#.reset_index(drop=True)
driver.close()

df

running search for 1 of 40 pages for data%20analyst.
total job links from page 1: 21
running search for 2 of 40 pages for data%20analyst.
total job links from page 2: 25
running search for 1 of 40 pages for data%20scientist.
total job links from page 1: 25
running search for 2 of 40 pages for data%20scientist.
total job links from page 2: 24
running search for 1 of 40 pages for data%20architect.
total job links from page 1: 25
running search for 2 of 40 pages for data%20architect.
total job links from page 2: 25
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find "full time" in job insights
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find "full time" in job insights
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find "full time" in job insights
Find another solution to SEE MORE on this

could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
could not find "full time" in job insights
could not find 'salary' with #SALARY tag
could not find 'salary' on p.t-16
could not find 'salary' in description
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
could not find "full time" in job insights
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
could not find "full time" in job insights
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
could not find "full time" in job insights
Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
c

Find another solution to SEE MORE on this page
could not find 'number of applicants' in applicant count
could not find 'number of applicants' in bullet
could not find "full time" in job insights
could not find 'salary' with #SALARY tag
could not find 'salary' on p.t-16
could not find 'salary' in description


,title,company,location,post_date,num_applicants,full_time,size,description,salary
0,Data Scientist,Frontiers,Spain,1 week ago,105 applicants,level,"1,001-5,000 employees · Book and Periodical Pu...",About the job\n \n\n \nWe are on a mi...,None
1,Senior Data Analyst (f/m),Axpo Group,"Madrid, Community of Madrid, Spain",14 hours ago,1 applicant,Associate,"5,001-10,000 employees · Utilities",About the job\n \n\n \n ...,None
2,Data Architect,Adobe,"Barcelona, Catalonia, Spain",1 week ago,56 applicants,level,"10,001+ employees · Software Development",About the job\n \n\n \nOur Company\n\...,None
3,Data scientist,HP,"Sant Cugat del Vallès, Catalonia, Spain",1 week ago,30 applicants,level,"10,001+ employees · IT Services and IT Consulting",About the job\n \n\n \n ...,None
4,Data Solution Architect,Bumble,"Barcelona, Catalonia, Spain",3 days ago,25 applicants,Associate,"501-1,000 employees · Technology, Information ...",About the job\n \n\n \nWe strongly en...,None
...,...,...,...,...,...,...,...,...,...
69,Principal Marketing Data Scientist,Bandai Namco Mobile,"Barcelona, Catalonia, Spain",2 weeks ago,None,level,11-50 employees · Computer Games,About the job\n \n\n \nPAC-MAN to Tek...,None
70,Data Platform Engineer (f/m/d),Axpo Group,"Madrid, Community of Madrid, Spain",1 day ago,None,Associate,"5,001-10,000 employees · Utilities",About the job\n \n\n \n ...,Full-time
71,Global Integration Services Architect,ADP,"Barcelona, Catalonia, Spain",1 day ago,None,Contract,"10,001+ employees · Human Resources Services",About the job\n \n\n \nGlobal Integra...,None
72,"Data scientist, Engineer, Physicist, Computer ...",German Aerospace Center (DLR),"Almería, Andalusia, Spain",1 month ago,None,level,"5,001-10,000 employees · Research Services",About the job\n \n\n \n ...,Part-time · Entry


### Notes for cleaning
- some bullet points don't separate words, so I will need to use regex to find key terms like 'Python' and 'Machine Learning.'
- skills and roles are mentioned multiple times throughout a description, so I will need to find a way to limit the skill count increase to one per page.

## Scrape Glassdoor using the Selenium Driver

In [9]:
# search glassdoor for all job titles
# save to glass_df: job title, salary range (or salary avg? both?)
# add glassdoor_salary column to df
# fill glassdoor_salary based on role name

## Export Data to CSV for Cleaning

In [16]:
df.to_csv('linkedin_jobs_uncleaned.csv', index = False, encoding='utf-8')